Solving Linear Regression with Gradient Descent using Tensorflow 2.0

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

W0727 18:11:09.656081 4523816384 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [3]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    version=1.0.0,
    description='The MNIST database of handwritten digits.',
    urls=['https://storage.googleapis.com/cvdf-datasets/mnist/'],
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [5]:
mnist_test

<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

Load the Data

In [6]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [7]:
test_data

<MapDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [8]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [9]:
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
validation_inputs, validation_targets = next(iter(validation_data))

test_data = test_data.batch(num_test_samples)

In [10]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
NUM_EPOCHS = 10
EARLY_STOPPING = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), callbacks=[EARLY_STOPPING], verbose=2)

Epoch 1/10
540/540 - 7s - loss: 0.0205 - accuracy: 0.9933 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
540/540 - 7s - loss: 0.0151 - accuracy: 0.9951 - val_loss: 0.0216 - val_accuracy: 0.9943
Epoch 3/10
540/540 - 7s - loss: 0.0163 - accuracy: 0.9943 - val_loss: 0.0276 - val_accuracy: 0.9917
Epoch 4/10
540/540 - 7s - loss: 0.0163 - accuracy: 0.9948 - val_loss: 0.0152 - val_accuracy: 0.9950
Epoch 5/10
540/540 - 7s - loss: 0.0125 - accuracy: 0.9957 - val_loss: 0.0113 - val_accuracy: 0.9968
Epoch 6/10
540/540 - 7s - loss: 0.0107 - accuracy: 0.9963 - val_loss: 0.0395 - val_accuracy: 0.9887
Epoch 7/10
540/540 - 7s - loss: 0.0152 - accuracy: 0.9947 - val_loss: 0.0190 - val_accuracy: 0.9940


In [54]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 1s 1s/step - loss: 0.0727 - accuracy: 0.9790

In [56]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.07. Test accuracy: 97.90%
